In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import gc
import os
import shutil

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")


In [ ]:
# Function to run inference on a single image
def infer_image(model, image_path, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0) # Add batch dimension
    image = image.to('cuda')
    
    with torch.no_grad():
        model.eval()
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()

In [ ]:
# Load your model (ensure it's on the correct device, e.g., CPU or GPU)
model = torch.load('./pth/test_ep25.pth')
model.to('cuda')

In [ ]:
# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def merge_files_to_directory(source_dir, target_dir):

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for subdir, _, files in os.walk(source_dir):
        for file in files:
            # Construct the full file path
            file_path = os.path.join(subdir, file)
            # Construct the target file path
            target_file_path = os.path.join(target_dir, file)

            # Check if file already exists in target directory
            if os.path.exists(target_file_path):
                print(f"File already exists: {target_file_path}")
                # Optionally, handle duplicates by renaming, skipping, or overwriting
            else:
                # Move or copy the file
                shutil.copy(file_path, target_file_path)  # Use shutil.copy() if you want to copy instead of move

    print(f"All files have been merged into {target_dir}")


In [ ]:
merge_files_to_directory("../../data/classifier/far_shah_b1-b3/test", "../../data/classifier/far_shah_b1-b3/inf")

In [ ]:
# Directory containing images
inf_dir = '../../data/classifier/far_shah_b1-b3/inf'

# List all files in the directory
image_paths = [os.path.join(inf_dir, f) for f in os.listdir(inf_dir) if os.path.isfile(os.path.join(inf_dir, f))]

In [ ]:
# Run inference on each image and store results
results = []
for image_path in image_paths:
    prediction = infer_image(model, image_path, transform)
    results.append((image_path, prediction))

In [ ]:
classes = sorted(os.listdir("../../data/classifier/far_shah_b1-b3/test"))
class_labels = {}
for i, one in enumerate(classes):
    class_labels.update({i: one})
    

In [ ]:
# Display a random subset of images with predictions
num_images_to_show = 5  # Set the number of images you want to display
display_indices = random.sample(range(len(results)), num_images_to_show)

plt.figure(figsize=(15, 5))
for i, idx in enumerate(display_indices, 1):
    image_path, pred_ind = results[idx]
    
    print(image_path, pred_ind)
    image = Image.open(image_path)

    plt.subplot(1, num_images_to_show, i)
    plt.imshow(image)
    plt.title(f"{class_labels[pred_ind]}")
    plt.axis('off')

plt.show()